In [1]:
import polars as pl
import numpy as np

In [2]:
epsilon     = 1.0  
sigma       = 1.0  
cutoff      = 2.5 * sigma 

In [8]:
label = 'T0.71_N7_RHO1'
positions_data = pl.read_csv(f'../output_files/{label}/{label}_positions_data.txt', separator=' ', new_columns=['particle', 'time', 'x', 'y', 'z'])

In [11]:
L = 11.1118

In [12]:
# Polars DataFrame to NumPy array for computation
positions = positions_data.select(['x', 'y', 'z']).to_numpy()

# NumPy array to store forces
forces = np.zeros_like(positions)

# Compute forces using NumPy
N = len(positions)  # Number of particles
for i in range(N):
    for j in range(i+1, N):
        # Compute distance accounting for periodic boundary conditions
        dx = positions[i, 0] - positions[j, 0]
        dy = positions[i, 1] - positions[j, 1]
        dz = positions[i, 2] - positions[j, 2]
        # Apply periodic boundary conditions (assuming cubic box with length 'L')
        dx -= L * np.round(dx / L)
        dy -= L * np.round(dy / L)
        dz -= L * np.round(dz / L)
        r = np.sqrt(dx**2 + dy**2 + dz**2)

        # Compute force if within cutoff distance
        if r < cutoff:
            # Lennard-Jones force calculation
            r6 = (sigma / r)**6
            r12 = r6**2
            force_magnitude = 24 * epsilon * (2 * r12 - r6) / r
            forces[i, 0] += force_magnitude * dx
            forces[i, 1] += force_magnitude * dy
            forces[i, 2] += force_magnitude * dz
            forces[j, 0] -= force_magnitude * dx  # Newton's third law
            forces[j, 1] -= force_magnitude * dy
            forces[j, 2] -= force_magnitude * dz

# Convert forces back to Polars DataFrame if needed
forces_df = pl.DataFrame(forces, columns=['forceX', 'forceY', 'forceZ'])

print(forces_df)

KeyboardInterrupt: 